<a href="https://colab.research.google.com/github/TLEtoAC/Prompt-Classification/blob/main/SLM_promptquality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pgrep -a ollama


41370 ollama serve


In [7]:
!ollama pull phi3:mini


In [8]:
!ollama run phi3:mini "Explain overfitting in machine learning"


Overfitting is a common problem that occurs in machine learning when a model learns the training data too well, including its noise and outliers. It essentially means that while the model performs exceptionally on this dataset due to these imperfections being learned as part of patterns, it fails significantly at predicting or generalizing results for new unseen datasets because such errors are not representative features but rather artifacts specific to the training data itself.

The primary reason overfitting occurs is that a model learns and memorizes too much detail from the training dataset without capturing its underlying patterns, leading it to perform poorly when presented with different information during testing or real-world application stages (i.e., validation/testing phase). This situation often happens due to highly complex models like deep neural networks which are overfit in nature because they have too many parameters relative to the number of observations available fo

In [11]:
import ollama
import sys
import re

# ===================================================================
# 1. THE CLASSIFIER (The "Brain")
# This is the 'phi3:mini' function we built.
# ===================================================================

# Define the "brain" for the classifier.
SYSTEM_PROMPT = """
You are a prompt classification router. Your only job is to analyze the user's
prompt and respond with a *single word*: low, medium, or high.
Do not explain your choice. Do not add any other text.
"""

# Create our "smarter" few-shot examples.
FEW_SHOT_EXAMPLES = [
    {'role': 'user', 'content': "What is 1 + 1?"},
    {'role': 'assistant', 'content': 'low'},
    {'role': 'user', 'content': "who was the president in 1999?"},
    {'role': 'assistant', 'content': 'low'},
    {'role': 'user', 'content': "what is the capital of france"},
    {'role': 'assistant', 'content': 'low'},
    {'role': 'user', 'content': "Why is microservices significant?"},
    {'role': 'assistant', 'content': 'medium'},
    {'role': 'user', 'content': "Explain quantum entanglement in simple terms."},
    {'role': 'assistant', 'content': 'medium'},
    {'role': 'user', 'content': "List the top 5 programming languages for machine learning."},
    {'role': 'assistant', 'content': 'medium'},
    {'role': 'user', 'content': "Summarize the following research paper abstract..."},
    {'role': 'assistant', 'content': 'high'},
    {'role': 'user', 'content': "Write a short story about a sad robot."},
    {'role': 'assistant', 'content': 'high'},
]


def predict_complexity(prompt_to_classify: str) -> str:
    """
    Uses the 'phi3:mini' model to classify a new prompt.
    """
    messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT}
    ]
    messages.extend(FEW_SHOT_EXAMPLES)
    messages.append({
        'role': 'user',
        'content': prompt_to_classify
    })

    try:
        # This will now connect to the 'ollama serve &' in Cell 3
        response = ollama.chat(
            model='phi3:mini',
            messages=messages,
            options={
                'num_ctx': 1024,
                'temperature': 0.0
            }
        )

        classification = response['message']['content'].lower().strip()

        words = re.split(r'[^a-z]+', classification)
        first_word = words[0] if words else ""

        if first_word in ["low", "medium", "high"]:
            return first_word

        print(f"Warning: Model gave unexpected output: '{classification}'", file=sys.stderr)
        return "medium" # Default to medium if confused

    except Exception as e:
        print(f"❌ Error during classification: {e}", file=sys.stderr)
        return "error"

# ===================================================================
# 2. THE ROUTER (The "Engine" Logic)
# This is the new "auto mode" function you asked for.
# ===================================================================

# Define our "model menu" for the carbon-aware router
MODEL_MENU = {
    "low": "gemma2:2b",         # Smallest, most efficient model
    "medium": "llama3:8b",    # Good, balanced local model
    "high": "gpt-4o-api"      # "Expensive" API model
}

def auto_route_prompt(user_prompt: str) -> str:
    """
    This is the main "auto mode" engine.
    1. It classifies the prompt.
    2. It selects the right model from the menu.
    """
    print(f"Prompt: \"{user_prompt}\"")

    # --- Step 1: Classify (using our 'phi3:mini' brain) ---
    complexity = predict_complexity(user_prompt)

    if complexity == "error":
        print("   >>> ERROR: Could not classify. Defaulting to medium.")
        return MODEL_MENU["medium"]

    print(f"   - Classified as: {complexity.upper()}")

    # --- Step 2: Route (the "auto mode" selection) ---
    selected_model = MODEL_MENU[complexity]
    print(f"   - Selected model: {selected_model}")

    # --- Step 3: Carbon-Logic (Your final goal) ---
    # This is where you will add your carbon-check
    if complexity == "high":
        # grid_is_clean = check_carbon_api() # <--- Your future function
        # if not grid_is_clean:
        #    print("   - Carbon grid is 'dirty', falling back to medium model.")
        #    selected_model = MODEL_MENU["medium"]
        pass # Placeholder for your carbon logic

    return selected_model

# ===================================================================
# 3. RUN THE ENGINE (Demo)
# ===================================================================

if __name__ == "__main__":

    prompts_to_test = [
        "What is 1 + 1?",
        "Explain quantum entanglement in simple terms.",
        "Compare the ethical implications of AI in healthcare vs autonomous vehicles.",
        "explain cholorophyll"
    ]

    print(f"--- Running Auto-Mode Selection Engine ---")

    for p in prompts_to_test:

        # This one function does it all
        selected_model = auto_route_prompt(p)

        # --- MOCK EXECUTION ---
        # This is where you would *actually* run the final prompt
        print(f"   >>> [MOCK] Would run: ollama.chat(model='{selected_model}', ...)")
        print("-" * 20)

--- Running Auto-Mode Selection Engine ---
Prompt: "What is 1 + 1?"
   - Classified as: LOW
   - Selected model: gemma2:2b
   >>> [MOCK] Would run: ollama.chat(model='gemma2:2b', ...)
--------------------
Prompt: "Explain quantum entanglement in simple terms."
   - Classified as: MEDIUM
   - Selected model: llama3:8b
   >>> [MOCK] Would run: ollama.chat(model='llama3:8b', ...)
--------------------
Prompt: "Compare the ethical implications of AI in healthcare vs autonomous vehicles."
   - Classified as: MEDIUM
   - Selected model: llama3:8b
   >>> [MOCK] Would run: ollama.chat(model='llama3:8b', ...)
--------------------
Prompt: "explain cholorophyll"
   - Classified as: LOW
   - Selected model: gemma2:2b
   >>> [MOCK] Would run: ollama.chat(model='gemma2:2b', ...)
--------------------
